## Conversion of CAMBOTv2 L0 natural-color (RGB) images
## to single-channel grayscale (luminance) images

To use ATM's CAMBOTv2 natural-color L0 (raw) images with the Ames Stereo Pipeline ([ASP](https://stereopipeline.readthedocs.io/en/latest/index.html)) they must be converted from RGB (red, green, blue) natural-color images to single-channel grayscale (luminance) images. The ASP user manual offers several ways to do this ([ASP RGB to gray](https://stereopipeline.readthedocs.io/en/latest/sfm.html#example-icebridge-dms-camera)). This Jupyter notebook demonstrates an alternative way for image conversion using the Pillow fork of the [Python Imaging Library](https://pillow.readthedocs.io/en/stable/). 

NASA's Airborne Topographic Mapper (ATM) CAMBOTv2 L0 raw images are freely available from the National Snow and Ice Data Center (data set ID: IOCAM0):  
https://nsidc.org/data/iocam0/versions/1  
DOI: https://doi.org/10.5067/IOJH8A5F48J5  
**NOTE:** Several example images are included in this repository in the folder `data/rgb`.

First, load the required Python™ modules and set up processing options and parameters.

In [1]:
import os
from PIL import Image

# set processing options
REPLACE = True    # overwrights existing luminance files if flag is set to True
RENAME  = False   # if True luminance files will have the string"_lum" added at the end of the file names

# set input and output directories for conversion
f_dir_cam = r".." + os.sep + "data"
f_dir_rgb = f_dir_cam + os.sep + "rgb"  # directpory with original RGB images
f_dir_lum = f_dir_cam + os.sep + "lum"  # directory with luminance (grayscale) converted images

# check if above directories exist and if not create directory for output files
if os.path.exists(f_dir_rgb):
    if os.path.exists(f_dir_lum) == False:
        os.makedirs(f_dir_lum)
        print("Created new folder for output files: {}".format(f_dir_lum))
else:
    os.sys.exit("Directory with CAMBOTv2 RGB files not found. Check directory paths.")

Now get list of images to convert and save converted images in different folder called `data/lum`.

In [2]:
# get list with names of CAMBOTv2 L0 .jpg files to convert
# r = root, d = directories, f = files

for r, d, f in os.walk(f_dir_rgb): 
    for file in f:
        if (file.startswith("IOCAM0") & file.endswith(".jpg")):
            f_name_inp = f_dir_rgb + os.sep + file
            f_name_out = f_dir_lum + os.sep + file
            f_name_tmp = os.path.splitext(f_name_out)
            if RENAME:
                f_name_lum = f_name_tmp[0] + "_lum" + f_name_tmp[1]
            else:
                f_name_lum = f_name_tmp[0] + f_name_tmp[1]
                
            del f_name_out, f_name_tmp
            
            if os.path.exists(f_name_lum) == False or REPLACE == True:
                print("Converting file: {}".format(file))
                # open the RGB image
                image_rgb = Image.open(f_name_inp)
                # get EXIF info
                exif = image_rgb.info['exif']
                # convert the RGB image to grayscale
                image_gray =  image_rgb.convert("L") # L stands for luminance
                # save the grayscale image with EXIF info
                image_gray.save(f_name_lum, exif=exif)

Converting file: IOCAM0_2019_GR_NASA_20190906-132014.4217.jpg
Converting file: IOCAM0_2019_GR_NASA_20190906-132014.9217.jpg
Converting file: IOCAM0_2019_GR_NASA_20190906-132015.4217.jpg
Converting file: IOCAM0_2019_GR_NASA_20190906-132015.9217.jpg
Converting file: IOCAM0_2019_GR_NASA_20190906-132016.4217.jpg
Converting file: IOCAM0_2019_GR_NASA_20190906-132016.9217.jpg
Converting file: IOCAM0_2019_GR_NASA_20190906-132017.4218.jpg
Converting file: IOCAM0_2019_GR_NASA_20190906-132017.9217.jpg
